In [1]:
import jax
import jax.nn as jnn
import jax.numpy as jnp
import numpy as np

import equinox as eqx
import diffrax

In [2]:
from policy.policy_training import data_generation, vmap_rollout_traj_env, vmap_compute_loss
from exciting_environments.pmsm.pmsm_env import PMSM

2024-10-27 12:34:13.357578: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.6.77). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
motor_env = PMSM(
    saturated=True,
    LUT_motor_name="BRUSA",
    batch_size=1,
    control_state=[])


In [4]:
data_batch_size=20
keys=jax.vmap(jax.random.PRNGKey)(np.random.randint(0, 2**31, size=(data_batch_size,)))
#key=jax.vmap(jax.random.PRNGKey)(jnp.linspace(0,1000,1001).astype(int))

In [5]:
in_obs,ref_ob,keys=data_generation(motor_env,keys)

In [6]:
def policy_fun(*args):
    return jnp.array([0.03,0.03])

In [7]:
def featurize(obs,ref_obs):
    return obs

In [8]:
obs,acts=vmap_rollout_traj_env(init_obs=in_obs,ref_obs=ref_ob,env=motor_env,horizon_length=15,policy=policy_fun,featurize=featurize)

In [9]:
def mse_loss(obs,ref_obs):
    loss=jnp.mean(jnp.sum((obs[:,:2]-ref_obs[:2])**2,axis=1))
    return loss

In [10]:
def penalty_loss(obs,ref_obs):
    loss=0
    return loss

In [11]:
vmap_compute_loss(obs,ref_ob,mse_loss,penalty_loss,weighting=1)

Array(624.619, dtype=float32)